In [7]:
from torch import nn
from torchvision import transforms, datasets
from torch.utils.data import random_split, DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence,pack_padded_sequence
import pandas as pd
import numpy as np
import torch
import tiktoken

In [8]:
import matplotlib.pyplot as plt
from tqdm import tqdm

import os
import csv

In [9]:
class MultiHeadAttention(nn.Module):
    def __init__(self, in_dim, out_dim, num_heads, qkv_bias=False,dropout=0.5):
        super().__init__()
        assert out_dim % num_heads == 0, "out_dim must be divisible by num_heads"

        self.out_dim = out_dim
        self.num_heads = num_heads
        self.head_dim = out_dim // num_heads
        self.W_query = nn.Linear(in_dim, out_dim, bias=qkv_bias)
        self.W_key = nn.Linear(in_dim, out_dim, bias=qkv_bias)
        self.W_value = nn.Linear(in_dim, out_dim, bias=qkv_bias)
        self.out_proj = nn.Linear(out_dim, out_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        batch_size, num_tokens, d_in = x.shape

        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        keys = keys.view(batch_size, num_tokens, self.num_heads, self.head_dim).transpose(1, 2)
        queries = queries.view(batch_size, num_tokens, self.num_heads, self.head_dim).transpose(1, 2)
        values = values.view(batch_size, num_tokens, self.num_heads, self.head_dim).transpose(1, 2)

        attn_scores = queries @ keys.transpose(-2, -1)

        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask, -float('inf'))

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        context_vec = (attn_weights @ values).transpose(1,2)
        
        context_vec = context_vec.contiguous().view(batch_size, num_tokens, self.out_dim)
        context_vec = self.out_proj(context_vec)

        return context_vec

In [10]:
batch = torch.randn(2, 4, 4)
batch_size, num_tokens, d_in = batch.shape
attn = MultiHeadAttention(in_dim=d_in, out_dim=8, num_heads=2)
out = attn(batch)

print(out.shape)
print(out)

torch.Size([2, 4, 8])
tensor([[[ 0.2556, -0.1940,  0.2607,  0.1174, -0.0680,  0.2049, -0.3367,
           0.2097],
         [ 0.4468, -0.0491,  0.3273,  0.1423, -0.2078,  0.3234, -0.1517,
           0.1535],
         [ 0.4373, -0.0391,  0.3073,  0.1436, -0.2163,  0.2824, -0.1676,
           0.1558],
         [ 0.4053, -0.1784,  0.3033,  0.0335, -0.1535,  0.1420, -0.1834,
           0.3192]],

        [[ 0.3602, -0.3550,  0.3689, -0.0370, -0.1272,  0.1849, -0.3959,
           0.6519],
         [ 0.4301, -0.1839,  0.3230,  0.0212, -0.2426,  0.1365, -0.3862,
           0.5303],
         [ 0.2827,  0.0107,  0.2738, -0.1564, -0.2311, -0.0395, -0.2515,
           0.2623],
         [ 0.6229, -0.1565,  0.4077, -0.0270, -0.3543,  0.1376, -0.2692,
           0.6404]]], grad_fn=<ViewBackward0>)


In [11]:
def collate_fn_imdb(data, pad_value=50257):
    data.sort(key=lambda x: len(x[0]), reverse=True)
    sequences = [x[0] for x in data]
    scores = torch.tensor([x[1] for x in data], dtype=torch.float32)
    labels = torch.tensor([x[2] for x in data], dtype=torch.float32)

    original_seq_lengths = torch.tensor([len(s) for s in sequences], dtype=torch.long)
    padded_seqs_long = pad_sequence(sequences, batch_first=True, padding_value=pad_value)

    return padded_seqs_long, original_seq_lengths, scores, labels

def tokenize_text(text_list, tokenizer):
    tokenized_text = []
    for text in text_list:
        tokens = torch.tensor(tokenizer.encode(text))
        tokenized_text.append(tokens)

    return tokenized_text

def detokenize_text(token_ids, tokenizer):
    detokenized_text = []
    for tokens in token_ids:
        text = tokenizer.decode(tokens.tolist())
        detokenized_text.append(text)

    return detokenized_text

class IMDBDataset(Dataset):
    def __init__(self, comments_token_ids, sentiments, scores):
        self.comments_token_ids = comments_token_ids
        self.sentiments = sentiments
        self.scores = scores

    def __len__(self):
        return len(self.comments_token_ids)
    
    def __getitem__(self, idx):
        return self.comments_token_ids[idx], self.scores[idx], self.sentiments[idx]
    
def create_IMDB_dataloader(dataset, batch_size=32, shuffle=True, num_workers=0, pad_value=50257):
    collate_wrapper = lambda x: collate_fn_imdb(x, pad_value=pad_value)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, collate_fn=collate_wrapper)
    return dataloader

In [ ]:
IMDB_train = pd.read_csv("data/IMDB_train.csv")
IMDB_test = pd.read_csv("data/IMDB_test.csv")

train_comments = IMDB_train["preprocessed_comments"].to_list()
train_sentiments = IMDB_train["sentiment"].to_list()
train_scores = IMDB_train["score"].to_list()
test_comments = IMDB_test["preprocessed_comments"].to_list()
test_sentiments = IMDB_test["sentiment"].to_list()
test_scores = IMDB_test["score"].to_list()
tokenizer = tiktoken.get_encoding("gpt2")

tokenized_train_comments = tokenize_text(train_comments, tokenizer)
tokenized_test_comments = tokenize_text(test_comments, tokenizer)
sample = train_comments[0]
sample_sentiment = train_sentiments[0]
sample_score = train_scores[0]
token_ids = tokenized_train_comments[0]
reconstructed = tokenizer.decode(token_ids.tolist())

print(f"Sample: {sample}")
print(f"Sentiment: {sample_sentiment}")
print(f"Score: {sample_score}")
print(f"Token IDs: {token_ids}")
print(f"Reconstructed: {reconstructed}")

FileNotFoundError: [Errno 2] No such file or directory: 'autodl-tmp/IMDB_train.csv'

In [ ]:
class GRUAttention(nn.Module):
    def __init__(self, in_dim, out_dim, num_heads, max_seq_len=1500, qkv_bias=False, dropout=0.5, vocab_size=50257, padding=50257):
        super().__init__()
        self.padding_idx = padding
        self.token_embedding = nn.Embedding(vocab_size + 1, in_dim, padding_idx=padding)
        self.pos_embedding = nn.Embedding(max_seq_len, in_dim)
        self.attention = MultiHeadAttention(in_dim, out_dim, num_heads, qkv_bias, dropout)
        self.relu1 = nn.ReLU()
        self.gru = nn.GRU(out_dim, out_dim//2, num_layers=1, batch_first=True)
        self.relu2 = nn.ReLU()
        self.fc = nn.Linear(out_dim//2, 1) 

    def forward(self, x):
        batch_size, num_tokens = x.shape

        token_embeddings = self.token_embedding(x)
        positions = torch.arange(num_tokens, device=x.device).unsqueeze(0).expand(batch_size, -1)
        pos_embeddings = self.pos_embedding(positions)
        input_embeddings = token_embeddings + pos_embeddings

        attn_mask = (x == self.padding_idx).unsqueeze(1).unsqueeze(2)
        
        attn_output = self.attention(input_embeddings, mask=attn_mask)
        attn_output = self.relu1(attn_output)
        
        gru_output, _ = self.gru(attn_output)
        pooled_gru_output = gru_output.mean(dim=1) 
        result = self.fc(pooled_gru_output)

        return result.squeeze(-1)
    
def train(model, train_dataset, val_dataset, lr=1e-3, epochs=10, batch_size=64):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()

    train_loader = create_IMDB_dataloader(train_dataset, batch_size=batch_size)
    val_loader = create_IMDB_dataloader(val_dataset, batch_size=batch_size)
    model.to(device)

    best_val_loss = float('inf')

    log_dir = "training_logs"
    os.makedirs(log_dir, exist_ok=True)
    log_file_path = os.path.join(log_dir, "training_metrics.csv")

    with open(log_file_path, 'w', newline='') as csvfile:
        fieldnames = ['Epoch', 'Train_Loss', 'Val_Loss', 'Val_Accuracy']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for epoch in range(epochs):
            model.train()
            total_loss = 0.0
            for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
                inputs, seq_lengths, scores, labels = batch
                inputs = inputs.to(device)
                scores = scores.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()
                outputs = model(inputs)
                loss = loss_fn(outputs, scores)
                total_loss += loss.item()
                loss.backward()
                optimizer.step()
            avg_loss = total_loss / len(train_loader)
            print(f"Epoch {epoch+1}/{epochs}, Train Loss(MSE): {avg_loss:.4f}")

            model.eval()
            val_loss = 0.0
            correct_sentiments = 0
            total_sentiments_samples = 0
            with torch.no_grad():
                for batch in tqdm(val_loader, desc="Validation"):
                    inputs, seq_lengths, scores, labels = batch
                    inputs = inputs.to(device)
                    scores = scores.to(device)
                    labels = labels.to(device)

                    outputs = model(inputs)
                    loss = loss_fn(outputs, scores)
                    val_loss += loss.item()
                    predicted_sentiments = (outputs > 5).float()
                    correct_sentiments += (predicted_sentiments == labels).sum().item()
                    total_sentiments_samples += labels.size(0)
            accuracy = correct_sentiments / total_sentiments_samples
            avg_val_loss = val_loss / len(val_loader)
            print(f"Epoch {epoch+1}/{epochs}, Validation Loss(MSE): {avg_val_loss:.4f}, Sentiment Accuracy: {accuracy:.4f}")
            
            writer.writerow({
                'Epoch': epoch + 1,
                'Train_Loss': avg_loss,
                'Val_Loss': avg_val_loss,
                'Val_Accuracy': accuracy
            })
            csvfile.flush()
            os.fsync(csvfile.fileno())

            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                os.makedirs("model", exist_ok=True)
                torch.save(model.state_dict(), "model/best_model.pth")
                print(f"Best model saved with validation loss: {best_val_loss:.4f}")

def test(model, test_dataset, batch_size=64):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    loss_fn = nn.MSELoss()

    test_loader = create_IMDB_dataloader(test_dataset, batch_size=batch_size, shuffle=False)
    model.to(device)
    model.eval()

    total_loss = 0.0
    correct_sentiments = 0
    total_sentiments_samples = 0

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Testing"):
            inputs, seq_lengths, scores, labels = batch
            inputs = inputs.to(device)
            scores = scores.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = loss_fn(outputs, scores)
            total_loss += loss.item()

            predicted_sentiments = (outputs > 5).float()
            correct_sentiments += (predicted_sentiments == labels).sum().item()
            total_sentiments_samples += labels.size(0)

    avg_loss = total_loss / len(test_loader)
    accuracy = correct_sentiments / total_sentiments_samples

    print(f"Test Loss(MSE): {avg_loss:.4f}, Test Sentiment Accuracy: {accuracy:.4f}")

In [ ]:
train_dataset = IMDBDataset(tokenized_train_comments, train_sentiments, train_scores)
test_dataset = IMDBDataset(tokenized_test_comments, test_sentiments, test_scores)

train_size = int(0.9 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

In [ ]:
model = GRUAttention(in_dim=64, out_dim=64, num_heads=4, max_seq_len=2048, qkv_bias=False, dropout=0.5)

train(model, train_dataset, val_dataset, lr=1e-4, epochs=50, batch_size=32)

Epoch 1/50: 100%|██████████| 704/704 [00:06<00:00, 102.99it/s]


Epoch 1/50, Train Loss(MSE): 19.2020


Validation: 100%|██████████| 79/79 [00:00<00:00, 327.88it/s]


Epoch 1/50, Validation Loss(MSE): 12.6444, Sentiment Accuracy: 0.5052
Best model saved with validation loss: 12.6444


Epoch 2/50: 100%|██████████| 704/704 [00:06<00:00, 104.34it/s]


...

Epoch 49/50, Validation Loss(MSE): 5.6202, Sentiment Accuracy: 0.8560


Epoch 50/50: 100%|██████████| 704/704 [00:06<00:00, 107.35it/s]


Epoch 50/50, Train Loss(MSE): 1.4805


Validation: 100%|██████████| 79/79 [00:00<00:00, 331.22it/s]

Epoch 50/50, Validation Loss(MSE): 5.6330, Sentiment Accuracy: 0.8548


In [ ]:
model.load_state_dict(torch.load("model/best_model.pth",weights_only=True))
test(model, test_dataset, batch_size=16)

Testing: 100%|██████████| 1563/1563 [00:02<00:00, 601.01it/s]

Test Loss(MSE): 5.9147, Test Sentiment Accuracy: 0.8358
